In [28]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
import os
from dotenv import load_dotenv

load_dotenv(".env")

True

In [29]:
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=os.getenv("AZURE_DEPLOYMENT"),
    model=os.getenv("MODE_NAME"),
    api_version=os.getenv("API_VERSION"),
    azure_endpoint=os.getenv("AZURE_ENDPOINT"),
    api_key=os.getenv("API_KEY"),
)

In [30]:

from autogen_core.models import UserMessage
result = await az_model_client.create(
    [UserMessage(content="What is the capital of France?", source="user")]
)
print(result)

finish_reason='stop' content='The capital of France is Paris.' usage=RequestUsage(prompt_tokens=15, completion_tokens=7) cached=False logprobs=None thought=None


In [31]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

In [32]:
async def save_blog_agent(blog_content: str) -> str:

    print("This is Code Interpreter for Azure AI Agent Service .......")


    code_interpreter = CodeInterpreterTool()
        
    agent = project_client.agents.create_agent(
            model="gpt-4o-mini",
            name="my-agent",
            instructions="You are helpful agent",
            tools=code_interpreter.definitions,
            # tool_resources=code_interpreter.resources,
    )

    thread = project_client.agents.create_thread()

    message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content="""
        
                    You are my Python programming assistant. Generate code,save """+ blog_content +
                    
                """    
                    and execute it according to the following requirements

                    1. Save blog content to blog-{YYMMDDHHMMSS}.md

                    2. give me the download this file link
                """,
    )
    # create and execute a run
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
            # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

        # # delete the original file from the agent to free up space (note: this does not delete your version of the file)
        # project_client.agents.delete_file(file.id)
        # print("Deleted file")

        # print the messages from the agent
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

        # get the most recent message from the assistant
    last_msg =  messages.get_last_text_message_by_role("assistant")
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")

        # print(f"File: {messages.file_path_annotations}")


    for file_path_annotation in messages.file_path_annotations:

        file_name = os.path.basename(file_path_annotation.text)

        project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./blog")
        

    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")


        # project_client.close()


    return "Saved"


    
    

In [33]:
from autogen_ext.agents.web_surfer import MultimodalWebSurfer

bing_search_agent = MultimodalWebSurfer(
        name="web_surfer",
        model_client=az_model_client,
        start_page="https://www.google.com",
    )

In [34]:
save_blog_content_agent = AssistantAgent(
    name="save_blog_content_agent",
    model_client=az_model_client,
    tools=[save_blog_agent],
    system_message="""
        Save blog content. Respond with 'Saved' to when your blog are saved.
    """
)

In [35]:
write_agent = AssistantAgent(
    name="write_agent",
    model_client=az_model_client,
    system_message="""
        You are a blog writer, please help me write a blog based on bing search content."
    """
)


In [36]:
text_termination = TextMentionTermination("Saved")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(10)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination

In [37]:

reflection_team = RoundRobinGroupChat([bing_search_agent, write_agent,save_blog_content_agent], termination_condition=termination)

In [38]:
await Console(
    reflection_team.run_stream(task="""

                    I am writing a blog about machine learning. Search for the following 3 questions and write a Chinese blog based on the search results ,save it
                    
                    1. What is Machine Learning?
                    2. The difference between AI and ML
                    3. The history of Machine Learning
                               

    """)
)  # Stream the messages to the console.

---------- user ----------


                    I am writing a blog about machine learning. Search for the following 3 questions and write a Chinese blog based on the search results ,save it
                    
                    1. What is Machine Learning?
                    2. The difference between AI and ML
                    3. The history of Machine Learning
                               

    
---------- web_surfer ----------
I typed 'What is Machine Learning?' into the browser search bar.

The web browser is open to the page [What is Machine Learning? - Search](https://www.bing.com/search?q=What+is+Machine+Learning%3F&FORM=QBLH).
The viewport shows 23% of the webpage, and is positioned at the top of the page
The following text is visible in the viewport:

Skip to content
What is Machine Learning?Deep search
All
Copilot
Images
Videos
Maps
More
Tools
Copilot Answer
GeeksForGeeks
https://www.geeksforgeeks.org › ml-machine-learningWhat is Machine Learning? - GeeksforGeeksMac

/workspaces/k8s-game-generator/venv/lib/python3.13/site-packages/msal/token_cache.py:164: DeprecationWarning: Use list(search(...)) instead to explicitly get a list.
  warnings.warn(


Run finished with status: RunStatus.COMPLETED
Messages: {'object': 'list', 'data': [{'id': 'msg_9aKvU9DguzAJbsQU5T3umy0B', 'object': 'thread.message', 'created_at': 1740704243, 'assistant_id': 'asst_dydQDcfR0gUtEPPqAZFOIITO', 'thread_id': 'thread_oDsoj88VYPs1Gyqo5ORzjIuC', 'run_id': 'run_Tjd37VynF2PN416FNBF6yUNA', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'The blog content has been saved successfully. You can download it using the link below:\n\n[Download the blog file](sandbox:/mnt/data/blog-250228005723.md)', 'annotations': [{'type': 'file_path', 'text': 'sandbox:/mnt/data/blog-250228005723.md', 'start_index': 114, 'end_index': 152, 'file_path': {'file_id': 'assistant-NF1uGdRt1JwJCy6NNACyNy'}}]}}], 'attachments': [{'file_id': 'assistant-NF1uGdRt1JwJCy6NNACyNy', 'tools': [{'type': 'code_interpreter'}]}], 'metadata': {}}, {'id': 'msg_4EeEogYQEMhmeYaSwRXVOjo7', 'object': 'thread.message', 'created_at': 1740704233, 'assistant_id': None, 'thread_id': 'thread_oDso

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='\n\n                    I am writing a blog about machine learning. Search for the following 3 questions and write a Chinese blog based on the search results ,save it\n                    \n                    1. What is Machine Learning?\n                    2. The difference between AI and ML\n                    3. The history of Machine Learning\n                               \n\n    ', type='TextMessage'), MultiModalMessage(source='web_surfer', models_usage=RequestUsage(prompt_tokens=2533, completion_tokens=116), content=['I typed \'What is Machine Learning?\' into the browser search bar.\n\nThe web browser is open to the page [What is Machine Learning? - Search](https://www.bing.com/search?q=What+is+Machine+Learning%3F&FORM=QBLH).\nThe viewport shows 23% of the webpage, and is positioned at the top of the page\nThe following text is visible in the viewport:\n\nSkip to content\nWhat is Machine Learning?De